In [1]:
# coding: utf-8

# Import all libraries needed for the tutorial
# General syntax to import specific functions in a library: 
##from (library) import (specific library function)
from pandas import DataFrame, read_csv
# General syntax to import a library but no functions: 
##import (library) as (give the library a nickname/alias)
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number
import functools
# from spectrum import *
from pylab import plot
from scipy import signal
import numpy as np
import time
from scipy import stats
# Enable inline plotting
from datetime import timedelta, date


#Clean Cells That Have only 0 values for 
def CleanNullCells(ZoneDataFrame):
    for kpi in column_kpiselection:
        for cell in ZoneDataFrame['CellName'].unique():
            CelluleData = ZoneDataFrame.loc[ZoneDataFrame['CellName']==cell][['DAY_',kpi]]
            if(CelluleData[kpi] == 0).all():
                  ZoneDataFrame = ZoneDataFrame.loc[ZoneDataFrame['CellName'] != cell]
    return(ZoneDataFrame) 


from datetime import timedelta, date, datetime

def KSTestPerDay(cell1,cell2,kpi):
    cell1_kpi = alldata_kpis.loc[alldata_kpis['CellName']==cell1][['DAY_',kpi]]
    cell1_kpi['DAY_'] = pd.to_datetime(cell1_kpi['DAY_'],infer_datetime_format=True)    
    cell2_kpi = alldata_kpis.loc[alldata_kpis['CellName']==cell2][['DAY_',kpi]]
    cell2_kpi['DAY_'] = pd.to_datetime(cell2_kpi['DAY_'],infer_datetime_format=True)
    
    KS = pd.DataFrame()
    for day in np.array([datetime(2018,8,7)+timedelta(days=i) for i in range(0,22)]):
        dayvalue = pd.DataFrame()
        dayvalue['Day'] = [day]
        mask1 = (cell1_kpi['DAY_'] >=day) & (cell1_kpi['DAY_'] <day+timedelta(days=1))
        mask2 = (cell2_kpi['DAY_'] >=day) & (cell2_kpi['DAY_'] <day+timedelta(days=1))

        dayvalue['pvalue'] = [stats.ks_2samp(cell1_kpi.loc[mask1][kpi],cell2_kpi.loc[mask2][kpi])[1]]
        KS = KS.append(dayvalue,ignore_index = True)
    return(KS)



from datetime import timedelta, date, datetime
def KSTestPerWeekDay(cell1,cell2,kpi):
    cell1_kpi = alldata_kpis.loc[alldata_kpis['CellName']==cell1][['DAY_',kpi]]   
    cell1_kpi['DAY_'] = pd.to_datetime(cell1_kpi['DAY_'],infer_datetime_format=True)
    cell2_kpi = alldata_kpis.loc[alldata_kpis['CellName']==cell2][['DAY_',kpi]]
    cell2_kpi['DAY_'] = pd.to_datetime(cell2_kpi['DAY_'],infer_datetime_format=True)
  
    KS = pd.DataFrame()
    for day in range(0,7):
        dayvalue = pd.DataFrame()
        dayvalue['Day'] = [day]
        mask1 = (cell1_kpi['DAY_'].dt.dayofweek==day)
        mask2 = (cell2_kpi['DAY_'].dt.dayofweek==day)
        dayvalue['pvalue'] = [stats.ks_2samp(cell1_kpi.loc[mask1][kpi],cell2_kpi.loc[mask2][kpi])[1]]
        KS = KS.append(dayvalue,ignore_index = True)
    return(KS)

def KSTestAllCellsPerDayAverage(ZoneDataFrame,kpi):
    KS = pd.DataFrame()
    alreadyDoneCouples = []
    for cell1 in ZoneDataFrame['CellName'].unique():
        for cell2 in ZoneDataFrame['CellName'].unique():
            if cell1 == cell2:
                KS.at[str(cell1),str(cell2)] = float(1)
            elif [cell2,cell1] in alreadyDoneCouples:
                KS.at[str(cell1),str(cell2)] = KS.loc[str(cell2)][str(cell1)]
            else:
                avg = KSTestPerDay(cell1,cell2,kpi)['pvalue'].mean()
                KS.at[cell1,cell2] = avg
                alreadyDoneCouples.append([cell1,cell2])
    return(KS)



def KSTestAllCellsPerDayStd(ZoneDataFrame,kpi):
    KS = pd.DataFrame()
    alreadyDoneCouples = []
    for cell1 in ZoneDataFrame['CellName'].unique():
        for cell2 in ZoneDataFrame['CellName'].unique():
            if cell1 == cell2:
                KS.loc[str(cell1),str(cell2)] = float(0)
            elif [cell2,cell1] in alreadyDoneCouples:
                KS.at[str(cell1),str(cell2)] = KS.loc[str(cell2)][str(cell1)]
            else:
                std = KSTestPerDay(cell1,cell2,kpi)['pvalue'].std()
                KS.at[str(cell1),str(cell2)] = float(std)
                alreadyDoneCouples.append([cell1,cell2])
    return(KS)



def KSTestAllCellsPerWeekDayAverage(ZoneDataFrame,kpi):
    KS = pd.DataFrame()
    alreadyDoneCouples = []
    for cell1 in ZoneDataFrame['CellName'].unique():
        for cell2 in ZoneDataFrame['CellName'].unique():
            if cell1 == cell2:
                KS.at[str(cell1),str(cell2)] = float(1)
            elif [cell2,cell1] in alreadyDoneCouples:
                KS.at[str(cell1),str(cell2)] = KS.loc[str(cell2)][str(cell1)]
            else:
                avg = KSTestPerWeekDay(cell1,cell2,kpi)['pvalue'].mean()
                KS.at[cell1,cell2] = avg
                alreadyDoneCouples.append([cell1,cell2])
    return(KS)


def KSTestAllCellsPerWeekDayStd(ZoneDataFrame,kpi):
    KS = pd.DataFrame()
    alreadyDoneCouples = []
    for cell1 in ZoneDataFrame['CellName'].unique():
        for cell2 in ZoneDataFrame['CellName'].unique():
            if cell1 == cell2:
                KS.loc[str(cell1),str(cell2)] = float(0)
            elif [cell2,cell1] in alreadyDoneCouples:
                KS.at[str(cell1),str(cell2)] = KS.loc[str(cell2)][str(cell1)]
            else:
                std = KSTestPerWeekDay(cell1,cell2,kpi)['pvalue'].std()
                KS.at[str(cell1),str(cell2)] = float(std)
                alreadyDoneCouples.append([cell1,cell2])
    return(KS)


# In[17]:


def getIdenticallyDistributedUnits(KSTestMatrix, alreadyDeletedCells = []): 
    count = 0
    totalpvalues = float(0)
    ToDeleteCell = 0
    for cell in KSTestMatrix.index.unique():
        A = np.array(KSTestMatrix[cell].values)
        A = A[np.where(A<0.1)]
        if len(A)>0:
            if count < len(A):
                ToDeleteCell = cell 
                count = len(A)
                totalpvalues = sum(A)
            elif (count == len(A)) & (totalpvalues < float(sum(A))):
                ToDeleteCell = cell 
                count = len(A)
                totalpvalues = sum(A)
    if ToDeleteCell == 0:
        return([KSTestMatrix,alreadyDeletedCells])
    else:
        alreadyDeletedCells.append(ToDeleteCell)
        FiltredMatrix = KSTestMatrix.drop([ToDeleteCell], axis = 0)
        FiltredMatrix = FiltredMatrix.drop([ToDeleteCell], axis = 1)
        return(getIdenticallyDistributedUnits(FiltredMatrix,alreadyDeletedCells))
        

# In[18]:


def getIdenticallyDistributedClusters(KSTestMatrix, partialResults=[]):
    Cluster = getIdenticallyDistributedUnits(KSTestMatrix,[])[0]
    DeletedCells = getIdenticallyDistributedUnits(KSTestMatrix,[])[1]
    partialResults.append(Cluster)
    if Cluster.size!=KSTestMatrix.size:
        Reste = KSTestMatrix.loc[DeletedCells][DeletedCells]
        return(getIdenticallyDistributedClusters(Reste,partialResults))
    else:
        return(partialResults)


# In[19]:


def generateLabeledDataFromClusters(ClusterList,kpi):
    LabeledData = pd.DataFrame()
    Label = 0
    for cluster in ClusterList:
        for clustercell in cluster.index:
            LabeledData.loc[str(clustercell),str(kpi)+' Label'] = Label
        Label = Label + 1
    return(LabeledData)


# In[20]:


def generateFeaturesFromCellList(CellList):
    FeaturesData = pd.DataFrame()
    for cell in CellList:
        FeaturesData.loc[cell,'Site'] = int(cell[1:len(cell)-1])
        if cell.startswith('D'):
            FeaturesData.loc[cell,'Tec'] = 0
            if cell[-1] =='A':
                FeaturesData.loc[cell,'Azimuth'] = 0
            elif cell[-1] =='B':
                FeaturesData.loc[cell,'Azimuth'] = 1
            elif cell[-1] =='C':
                FeaturesData.loc[cell,'Azimuth'] = 2    
        elif cell.startswith('L'):
            FeaturesData.loc[cell,'Tec'] = 1
            if cell[-1] =='A':
                FeaturesData.loc[cell,'Azimuth'] = 0
            elif cell[-1] =='B':
                FeaturesData.loc[cell,'Azimuth'] = 1
            elif cell[-1] =='C':
                FeaturesData.loc[cell,'Azimuth'] = 2
        elif cell.startswith('T'):
            FeaturesData.loc[cell,'Tec'] = 2
            if cell[-1] =='A':
                FeaturesData.loc[cell,'Azimuth'] = 0
            elif cell[-1] =='B':
                FeaturesData.loc[cell,'Azimuth'] = 1
            elif cell[-1] =='C':
                FeaturesData.loc[cell,'Azimuth'] = 2
    return(FeaturesData)


# In[21]:


def generateKpiLabeledDataFromExcelKSTEST(excelFileLocation, kpiList):
    allKpiDataFrame = pd.DataFrame()
    i = 0
    for kpi in kpiList:
        kpiKSTEST = pd.read_excel(excelFileLocation, sheet_name = kpi)
        kpiClusters = getIdenticallyDistributedClusters(kpiKSTEST,[])
        kpiLabeledData = generateLabeledDataFromClusters(kpiClusters,kpi)
        if i == 0:
            cellFeatures = generateFeaturesFromCellList(kpiKSTEST.index)
            allKpiDataFrame = cellFeatures.join(kpiLabeledData)
            i = i+1
        else:
            allKpiDataFrame = allKpiDataFrame.join(kpiLabeledData) 
    return(allKpiDataFrame)

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)


print('Python version ' + sys.version)
print('Pandas version ' + pd.__version__)
print('Matplotlib version ' + matplotlib.__version__)

Python version 3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]
Pandas version 0.23.0
Matplotlib version 2.2.2


In [2]:
#Parallel Computing Functions:
import multiprocessing  
import random
from multiprocessing import Pool
import KSTEST
import itertools
def KSTestAllCellsPerDayAverageParallel(ZoneDataFrame,kpi):
    if __name__ == '__main__':
        from datetime import timedelta, date, datetime
        pool = Pool()
        t = time.time()
        C = list(itertools.combinations(ZoneDataFrame['CellName'].unique(),2))
        A = [(ZoneDataFrame, c[0],c[1],kpi) for c in C]
        print(time.time()-t)
        results = pool.starmap(KSTEST.KSTestPerDayAverageParallelFunc, A)
        del A
    return(results)

def buildKSTESTMatrixFromMPTEST(result):
    KS = pd.DataFrame()
    for res in result:
        cell1 = res.index.values[0]
        cell2 = res.columns.values[0]
        pvalue = res.loc[res.index.values[0]][res.columns.values[0]] 
        KS.at[str(cell1),str(cell2)] = pvalue
        KS.at[str(cell2),str(cell1)] = pvalue
    KS = KS.fillna(1)
    return(KS)

Python version 3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]
Pandas version 0.23.0
Matplotlib version 2.2.2


In [3]:
column_selection = ["DAY_","Departement","SiteTheorique","eNodeBName","Secteur","CellName","CQImoyen_Convert","CROZON_DemandesVoix4G",
                    "CROZON_NombredeConnexionsPS","CROZON_SucceseRABData4G","CROZON_TraficDataDL4G","CROZON_SucceseRABPS4G","CoupuresData",
                    "CROZON_SucceseRABVoix4G","CSFB_Tot","CROZON_TDC4GData","CROZON_CoupureseRABData4G_old","CROZON_TDC4GData_Mobilite",
                    "CROZON_DemandesData4G","CROZON_TDC4GDataCluster","Badcoveragedetected_A2_","CROZON_TDC4GDataCluster_Mobilite",
                    "CROZON_CoupureseRABPS4G","CROZON_TDC4GPSCluster_Mobilite","CROZON_TDC4GPS","CQImoyen","CROZON_TDC4GPS_Mobilite",
                    "CROZON_TraficDataUL4G","CROZON_TDC4GPSCluster","ErabrelnormalQCI9","ErabRelQCi5","nbUEVoIPqualityOK","nbUEVoIPqualityNOK",
                    "Nbrtentatives_QCi5__TEAVoix","Nbrtentatives_QCi5__TEAVisio","Nbrtentatives_QCi1_","NbreConn_Voix","ErabrelnormalQCI5",
                    "ErabVoixRelease_Cellule","ErabrelnormalQCI8","NbHOPrepAttIFLB","ErabVoixRelease_Cluster","ErabrelnormalQCI6","NbdropsDataActifseNodeB",
                    "NbreConn_Data","NbHOExeAttIFLB","TBRe_transPUSCH","TBRetransPDSCH","TBtransPUSCH","SCTot","TBtransPDSCH","PingPong_HOinter_HOOscillation",
                    "TraficDataDL","PingPong_HOintra_HOOscillation","TraficDataUL","Traficvoix_Traficvoix_visio","VoIPqualitymeasinsuffstat"]


column_kpiselection = ["CQImoyen_Convert","CROZON_DemandesVoix4G",
                    "CROZON_NombredeConnexionsPS","CROZON_SucceseRABData4G","CROZON_TraficDataDL4G","CROZON_SucceseRABPS4G","CoupuresData",
                    "CROZON_SucceseRABVoix4G","CSFB_Tot","CROZON_TDC4GData","CROZON_TDC4GData_Mobilite",
                    "CROZON_DemandesData4G","CROZON_TDC4GDataCluster","CROZON_TDC4GDataCluster_Mobilite",
                    "CROZON_CoupureseRABPS4G","CROZON_TDC4GPSCluster_Mobilite","CROZON_TDC4GPS","CQImoyen","CROZON_TDC4GPS_Mobilite",
                    "CROZON_TraficDataUL4G","CROZON_TDC4GPSCluster","ErabrelnormalQCI9","ErabRelQCi5","nbUEVoIPqualityOK",
                    "Nbrtentatives_QCi5__TEAVoix","Nbrtentatives_QCi5__TEAVisio","Nbrtentatives_QCi1_","NbreConn_Voix","ErabrelnormalQCI5",
                    "ErabVoixRelease_Cellule","ErabrelnormalQCI8","NbHOPrepAttIFLB","ErabrelnormalQCI6",
                    "NbreConn_Data",
                    "TraficDataDL","TraficDataUL","Traficvoix_Traficvoix_visio"]

In [4]:
########################################
# import des donnees
########################################


start_date = date(2018, 8, 7)
end_date = date(2018, 8, 29)

#Regroupe data of all dates 1
datestring = end_date.strftime("%Y%m%d")
#Location1 =  str(data_path)+datestring+'_1.CSV'
Location1 =  str('C:/Users/euripab/Desktop/ML Extraction/Modified Files/')+datestring+'_1.CSV'
date_data = pd.read_csv(Location1, sep=';', encoding='latin-1')
data_1 = pd.read_csv(Location1, sep=';', encoding='latin-1')
data_1  = data_1[column_selection]
data_1 = data_1.replace(r'\N',np.nan)

for single_date in daterange(start_date, end_date):
    datestring = single_date.strftime("%Y%m%d")
    #Location1 =  str(data_path)+datestring+'_1.CSV'
    Location1 =  str('C:/Users/euripab/Desktop/ML Extraction/Modified Files/')+datestring+'_1.CSV'
    date_data = pd.read_csv(Location1, sep=';', encoding='latin-1')
    date_data = date_data[column_selection]
    date_data = date_data.replace(r'\N',np.nan)
    data_1 = data_1.append(date_data)


#Regroupe data of all dates 2
datestring = end_date.strftime("%Y%m%d")
#Location2 =  str(data_path)+datestring+'_2.CSV'
Location2 =  str('C:/Users/euripab/Desktop/ML Extraction/Modified Files/')+datestring+'_2.CSV'
date_data = pd.read_csv(Location2, sep=';', encoding='latin-1')
data_2 = pd.read_csv(Location2, sep=';', encoding='latin-1')
data_2  = data_2[column_selection]
data_2 = data_2.replace(r'\N',np.nan)
for single_date in daterange(start_date, end_date):
    datestring = single_date.strftime("%Y%m%d")
    #Location2 =  str(data_path)+datestring+'_2.CSV'
    Location2 =  str('C:/Users/euripab/Desktop/ML Extraction/Modified Files/')+datestring+'_2.CSV'
    date_data = pd.read_csv(Location2, sep=';', encoding='latin-1')
    date_data = date_data[column_selection]
    date_data = date_data.replace(r'\N',np.nan)
    data_2 = data_2.append(date_data)
    
    
#Create Final Data File
alldata_kpis = data_1.append(data_2)
del data_1
del data_2
print(alldata_kpis.dtypes)


C:\Users\euripab\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (625,626,627,628) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


DAY_                                object
Departement                          int64
SiteTheorique                       object
eNodeBName                          object
Secteur                             object
CellName                            object
CQImoyen_Convert                    object
CROZON_DemandesVoix4G                int64
CROZON_NombredeConnexionsPS          int64
CROZON_SucceseRABData4G              int64
CROZON_TraficDataDL4G               object
CROZON_SucceseRABPS4G                int64
CoupuresData                         int64
CROZON_SucceseRABVoix4G              int64
CSFB_Tot                             int64
CROZON_TDC4GData                    object
CROZON_CoupureseRABData4G_old        int64
CROZON_TDC4GData_Mobilite           object
CROZON_DemandesData4G                int64
CROZON_TDC4GDataCluster             object
Badcoveragedetected_A2_              int64
CROZON_TDC4GDataCluster_Mobilite    object
CROZON_CoupureseRABPS4G              int64
CROZON_TDC4

In [5]:
alldata_kpis[column_kpiselection] = alldata_kpis[column_kpiselection].astype(str).apply(lambda x: x.str.replace(',','.'))
alldata_kpis[column_kpiselection] = alldata_kpis[column_kpiselection].astype(float)
alldata_kpis = alldata_kpis.drop(['Departement','SiteTheorique','eNodeBName','Secteur'], axis = 1)

alldata_kpis = CleanNullCells(alldata_kpis)

In [6]:
def get_config(local_fic_conf, local_type):
    
    # print(local_type)
    liste_retour = []
    fic_config = open(local_fic_conf, "r")
    fin = 0 
    
    for line in fic_config:
        # print(line)
        line = line.replace("\n", "")        
        if line.find(local_type)>=0 and fin == 0 :
            # print("trouve")
            for line in fic_config :
                if line != "\n" and fin==0:
                    line = line.replace("\n", "")
                    liste_retour.append(line)
                else:
                    # print("fin trouve")
                    fin = 1
    fic_config.close()
    return(liste_retour)

def extract_list_from_file(local_file):
    loc_file = open(local_file, "r")
    data_read=[]
    for line in loc_file:
        line = line.replace("\n", "")
        data_read.append(line)
    loc_file.close()
    
    return (data_read)



###################
# global variables
###################
local_path = "C:/Users/euripab/Desktop/ML Extraction"
config_file = "config_python_max_jour.txt"

config_group = get_config(local_path + "/" + config_file, "config_group")

list_group=[]
inventaire_groupe=[]
inventaire_selected_struct=[]
if len(config_group)==1 :
    # extract group configuration
    list_group = extract_list_from_file(local_path + "/" + config_group[0])
    groupe_definition = {}
    for groupe_def in list_group :
        groupe_def2 = groupe_def.split(";")
        inventaire_selected_struct.append(groupe_def2[1])
        if not groupe_def2[0] in inventaire_groupe :
            inventaire_groupe.append(groupe_def2[0])
        groupe_definition[groupe_def2[1]] = groupe_def2[0]
        
# pour extraire les donnees de chaque groupe
# passer de liste a dataframe ()
array_list_group=[]
# pb : on garde les entetes dans les donnees -> a corriger
# les colonnes du dataframe ne sontpas nommees
for item in list_group:
    split_item = item.split(";")
    array_list_group.append(split_item)

df_list_group = pd.DataFrame(array_list_group)  

# extraction des donnees du groupe residentiel
df_list_group.loc[df_list_group[0]=='residentiel'][1]
ResCellsKpi = alldata_kpis.loc[alldata_kpis['CellName'].isin(df_list_group.loc[df_list_group[0]=='residentiel'][1])]
    
kpiList = ['CQImoyen','TraficDataDL',
           'VoIPqualitymeasinsuffstat','TraficDataUL']
    
#computing_type = "single" 
computing_type = "parallel"
working_path = r'C:/Users/euripab/Desktop/ML Extraction'
list_type_cell = ['L','D','T']

In [7]:
for current_groupe in ['residentiel']:
    # pour extraire les cellules du groupe residentiel
    df_list_group.loc[df_list_group[0]==current_groupe][1]

    # extraction des donnees du groupe residentiel
    # df_list_group.loc[df_list_group[0]=='residentiel'][1]
    ZoneCellsKpi = alldata_kpis.loc[alldata_kpis['CellName'].isin(df_list_group.loc[df_list_group[0]==current_groupe][1])]
    for type_cell in list_type_cell:
        ZoneCellTypeKpi = ZoneCellsKpi.loc[ZoneCellsKpi['CellName'].str.startswith(type_cell)]
        fic_result= working_path + "\KSTest_" + current_groupe +"_"+type_cell + ".xlsx"
        writer = pd.ExcelWriter(fic_result)
        #matrivce P_value
        #cas parallele computing
        if computing_type == 'single':
            for kpi in kpiList:
                KpiPvalueMatrix = KSTestAllCellsPerDayAverage(ZoneCellTypeKpi,kpi)
                KpiPvalueMatrix.to_excel(writer,sheet_name=kpi)
            writer.save()
        elif computing_type == 'parallel':
            for kpi in kpiList:
                results = KSTestAllCellsPerDayAverageParallel(ZoneCellTypeKpi,kpi)
                KpiPvalueMatrix = buildKSTESTMatrixFromMPTEST(results)
                KpiPvalueMatrix.to_excel(writer,sheet_name=kpi)
            writer.save()
        

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [8]:

for current_groupe in inventaire_groupe:
    # pour extraire les cellules du groupe residentiel
    df_list_group.loc[df_list_group[0]==current_groupe][1]

    # extraction des donnees du groupe residentiel
    # df_list_group.loc[df_list_group[0]=='residentiel'][1]
    SingleGroupCellsKpi = alldata_kpis.loc[alldata_kpis['CellName'].isin(df_list_group.loc[df_list_group[0]==current_groupe][1])]
    print("groupe = {}, taille = {}".format(current_groupe, len(ResCellsKpi)))

groupe = residentiel, taille = 14904
groupe = Gare_St_lazare, taille = 14904
groupe = Chatelet_les_halles, taille = 14904
groupe = rive_droite_seine, taille = 14904
groupe = concorde, taille = 14904


In [9]:
inventaire_groupe

['residentiel',
 'Gare_St_lazare',
 'Chatelet_les_halles',
 'rive_droite_seine',
 'concorde']

In [10]:
# df_list_group.loc[df_list_group[0]=='residentiel'][1]

ResCellsKpi = alldata_kpis.loc[alldata_kpis['CellName'].isin(df_list_group.loc[df_list_group[0]=='residentiel'][1])]

In [11]:
len(ResCellsKpi)

14904

In [12]:
df_list_group[0].unique()

array(['residentiel', 'Gare_St_lazare', 'Chatelet_les_halles',
       'rive_droite_seine', 'concorde'], dtype=object)

In [13]:
list_group

['residentiel;D10664A',
 'residentiel;L10664A',
 'residentiel;P10664A',
 'residentiel;T10664A',
 'residentiel;D10664B',
 'residentiel;L10664B',
 'residentiel;P10664B',
 'residentiel;T10664B',
 'residentiel;D10664C',
 'residentiel;L10664C',
 'residentiel;P10664C',
 'residentiel;T10664C',
 'residentiel;D12455A',
 'residentiel;D15843A',
 'residentiel;L15843A',
 'residentiel;P15843A',
 'residentiel;T15843A',
 'residentiel;D15843B',
 'residentiel;L15843B',
 'residentiel;P15843B',
 'residentiel;T15843B',
 'residentiel;D15843C',
 'residentiel;L15843C',
 'residentiel;P15843C',
 'residentiel;T15843C',
 'residentiel;D16785A',
 'residentiel;L16785A',
 'residentiel;T16785A',
 'residentiel;D16785B',
 'residentiel;L16785B',
 'residentiel;T16785B',
 'residentiel;D16785C',
 'residentiel;L16785C',
 'residentiel;T16785C',
 'Gare_St_lazare;D10545A',
 'Gare_St_lazare;L10545A',
 'Gare_St_lazare;P10545A',
 'Gare_St_lazare;T10545A',
 'Gare_St_lazare;D10545B',
 'Gare_St_lazare;L10545B',
 'Gare_St_lazare;P1054

In [91]:
KSTestPerDay('L10664C','D15843B','CQImoyen')['Day'][0]

Timestamp('2018-08-07 00:00:00')

In [102]:
def KSTestAllCellsPerDayAll(ZoneDataFrame,kpi):
    i = 0
    out = {}
    alreadyDoneCouples = []
    for cell1 in ZoneDataFrame['CellName'].unique():
        for cell2 in ZoneDataFrame['CellName'].unique():
            if cell1 == cell2:
                if i == 0:
                    for day in KSTestPerDay(cell1,cell2,kpi)['Day']:
                        out[day.date()] = pd.DataFrame()
                        out[day.date()].at[str(cell1),str(cell2)] = float(1)
                    i = i+1
                else:
                    for day in out:
                        out[day].at[str(cell1),str(cell2)] = float(1)
            elif [cell2,cell1] in alreadyDoneCouples:
                for day in out:
                    out[day].at[str(cell1),str(cell2)] = out[day].loc[str(cell2)][str(cell1)]
            else:
                A= KSTestPerDay(cell1,cell2,kpi)
                A = A.set_index('Day')
                for day in A.index:
                    out[day.date()].at[str(cell1),str(cell2)] = A.loc[day]['pvalue']
                alreadyDoneCouples.append([cell1,cell2])
    return(out)